In [1]:

import os

from rlxf.preference_dataset import PreferenceDataset
from rlxf.llm import LLM

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# Instantiate the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sshleifer/tiny-gpt2")
model = AutoModelForCausalLM.from_pretrained("sshleifer/tiny-gpt2")
#tokenizer = AutoTokenizer.from_pretrained("PY007/TinyLlama-1.1B-Chat-v0.3")
#model = AutoModelForCausalLM.from_pretrained("PY007/TinyLlama-1.1B-Chat-v0.3")

# Create a dataset
dataset = Dataset.from_dict(
    {"text": [
        "Write an email for B2B marketing: ##EMAIL: ", 
       # "What is the name of the capital of France? ",
    ]}
)

# Configure the LLM
llm = LLM(model=model, tokenizer=tokenizer, num_responses=2)

preference_dataset = PreferenceDataset(dataset, llm=llm)

# Execute methods for the PreferenceDataset
rated_dataset = preference_dataset.generate()

rg_dataset = preference_dataset.to_argilla()

/Users/danielvilasuero/Library/Caches/pypoetry/virtualenvs/rlxf-3J6KC4no-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map:   0%|          | 0/1 [00:00<?, ? examples/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Map: 100%|██████████| 1/1 [00:09<00:00,  9.82s/ examples]

[{'question_name': 'rating_0', 'value': 1, 'agent': 'gpt-4'}, {'question_name': 'rationale_0', 'value': 'The text is severely incorrect. It does not fulfill the task objective of writing an email for B2B marketing. The text is a random collection of words and phrases, making it incomprehensible and uninformative.', 'agent': 'gpt-4'}, {'question_name': 'rating_1', 'value': 1, 'agent': 'gpt-4'}, {'question_name': 'rationale_1', 'value': 'The text is severely incorrect. It does not fulfill the task objective of writing an email for B2B marketing. The text is a random collection of words and phrases, making it incomprehensible and uninformative.', 'agent': 'gpt-4'}]


In [2]:
import argilla as rg

rg.init(
    api_key="admin.apikey",
    api_url="https://argilla-metadata.hf.space"
)
rg_dataset.push_to_argilla(name="rlfx-demo", workspace="admin")

/Users/danielvilasuero/Library/Caches/pypoetry/virtualenvs/rlxf-3J6KC4no-py3.11/lib/python3.11/site-packages/argilla/client/client.py:165: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
Pushing records to Argilla...: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


<FeedbackDataset id=36d66895-e4e7-429f-8826-012faa15a2d3 name=rlfx-demo workspace=Workspace(id=864f1f19-079d-4a75-84bc-f6504465b7a5, name=admin, inserted_at=2023-10-15 05:02:29, updated_at=2023-10-15 05:02:29) url=https://argilla-metadata.hf.space/dataset/36d66895-e4e7-429f-8826-012faa15a2d3/annotation-mode fields=[TextField(id=UUID('8c40631e-0b8a-48ec-af4e-7cc603db8afe'), name='input', title='Input', required=True, type='text', settings={'type': 'text', 'use_markdown': False}, use_markdown=False), TextField(id=UUID('e8cca350-b162-4dcc-9f8e-200c4ea9c6ed'), name='response_0', title='Response_0', required=True, type='text', settings={'type': 'text', 'use_markdown': True}, use_markdown=True), TextField(id=UUID('4589a2c6-89fc-4f43-bdc7-050eea983351'), name='response_1', title='Response_1', required=True, type='text', settings={'type': 'text', 'use_markdown': True}, use_markdown=True)] questions=[RatingQuestion(id=UUID('fda7b156-006a-42dc-a2d0-328dc207e9da'), name='rating_0', title='Rate th

In [3]:
rated_dataset.to_pandas().rated_responses[1]

array([{'rating': '1', 'rationale': 'The text is severely incorrect. It does not provide any useful or relevant information in response to the task instruction. It seems to be a random collection of words and phrases, which makes it incomprehensible and uninformative.'},
       {'rating': '1', 'rationale': 'The text is severely incorrect. Similar to Text 1, it does not provide any useful or relevant information in response to the task instruction. It appears to be a random collection of words and phrases, making it incomprehensible and uninformative.'}],
      dtype=object)